This notebook describes a pipeline to download panoramas and their corresponding depth maps from Google Street View.  
It takes an input ```.json``` describing a list of EPSG:4326 latitude/longitude pairs (as a list of length 2), and attempts to save the panorama as a ```.jpg``` and a 3D cartesian point cloud derived from the depth map as a ```.json``` in the target directory.  
A ```.csv``` indicating the panorama IDs (which correspond to the filenames) with the true latitude, longitude, and elevation is also saved.

In [1]:
target_dir = "data"
input_latlons_fp = "place-pulse-singapore-latlons.json"
pano_dir = "panos"
point_cloud_dir = "point-clouds"
log_filename = "street-view-id.csv"

In [2]:
import aiocsv
import aiofiles
from aiohttp import ClientSession, ClientTimeout
from streetlevel import streetview

import asyncio
import csv
import json
import math
from pathlib import Path
import os

In [3]:
def unique_latlon(latlons: list[list[float]]) -> list[list[float]]:
    unique_latlons = set()
    for latlon in latlons:
        unique_latlons.add(tuple(latlon))
    return [list(x) for x in unique_latlons]

def depthmap_to_xyz(depthmap: list[list[float]],
                    xrange: tuple[float, float] = (-1.0, 1.0), yrange: tuple[float, float] = (-1.0, 1.0),
                    heading: float = 0, rfilter=math.inf) -> list[list[float]]:
    pi = math.pi
    sin = math.sin
    cos = math.cos
    output = []
    width = len(depthmap[0])
    height = len(depthmap)
    x0 = xrange[0]
    dx = xrange[1] - x0
    y0 = yrange[0]
    dy = yrange[1] - y0
    h = -heading
    for i in range(height):
        for j in range(width):
            r = depthmap[i][j]
            if r > rfilter:
                continue
            xnorm = ((j + 0.5) / width) * dx + x0
            ynorm = ((i + 0.5) / height) * dy + y0
            theta = -pi * xnorm
            phi = -pi / 2 * ynorm
            cartesian = [[-r * sin(h + theta) * cos(phi)],
                         [r * cos(h + theta) * cos(phi)],
                         [r * sin(phi)]]
            output.append([cartesian[0][0], cartesian[1][0], cartesian[2][0]])
    return output

In [4]:
async def find_pano_id(session, lat, lon, radius=50):
    try:
        return await streetview.find_panorama_async(lat, lon, session, radius=radius)
    except Exception as e:
        print(f"find_pano_id, ({lat}, {lon}): e")
        return None

async def find_pano_depth(session, panoid):
    try:
        return await streetview.find_panorama_by_id_async(panoid, session, download_depth=True)
    except Exception as e:
        print(f"find_pano_depth, ({panoid}): e")
        return None
    
async def find_pano_full(session, lat, lon, radius=50):
    base_pano = await find_pano_id(session, lat, lon, radius)
    if base_pano:
        return await find_pano_depth(session, base_pano.id)
    else:
        return None

async def save_pano_point_cloud(target_dir, pano_dir, point_cloud_dir, lat, lon, session, csv_writer):
    pano = await find_pano_full(session, lat, lon)
    if pano and pano.depth and pano.heading and pano.elevation:
        try:
            image = await streetview.get_panorama_async(pano, session, zoom=0)
            point_cloud = await asyncio.to_thread(depthmap_to_xyz, pano.depth.data[:,::-1], heading=pano.heading, rfilter=60)
            await csv_writer.writerow([pano.id, pano.lat, pano.lon, pano.elevation])
            await asyncio.to_thread(image.save, os.path.join(target_dir, pano_dir, f"{pano.id}.jpg"))
            async with aiofiles.open(os.path.join(target_dir, point_cloud_dir, f"{pano.id}.json"), mode='w') as fp:
                await fp.write(json.dumps(point_cloud))
        except Exception as e:
            print(f"save_pano_point_cloud, {pano.id}: e")

async def main(latlons, target_dir, pano_dir, point_cloud_dir, log_filename):
    Path(os.path.join(target_dir, pano_dir)).mkdir(parents=True, exist_ok=True)
    Path(os.path.join(target_dir, point_cloud_dir)).mkdir(parents=True, exist_ok=True)
    async with ClientSession() as session:
        async with aiofiles.open(os.path.join(target_dir, log_filename), mode='w+') as fp:
            csv_writer = aiocsv.AsyncWriter(fp)
            await csv_writer.writerow(["id", "lat", "lon", "elevation"])
            tasks = [save_pano_point_cloud(target_dir, pano_dir, point_cloud_dir, latlon[0], latlon[1], session, csv_writer) for latlon in latlons]
            await asyncio.gather(*tasks)

In [5]:
input_latlons = []
with open(os.path.join(target_dir, input_latlons_fp), 'r') as fp:
    input_latlons = json.load(fp)
input_latlons = unique_latlon(input_latlons)

await main(input_latlons, target_dir, pano_dir, point_cloud_dir, log_filename)

find_pano_depth, (EH0KBgr7HPM2O1rd-t67Ig): e
find_pano_depth, (VTQz6NTFgqshz5bMNT9PtQ): e
find_pano_depth, (TTMg3SOBxTasR2OvCzPZxw): e
find_pano_depth, (Eno-NZ-Qk05HllVKlJ6JkQ): e


ServerDisconnectedError: Server disconnected

find_pano_depth, (nPkCJIeUvkV55ROfwUto8g): e
find_pano_depth, (TM7TQLzwONFstjQJEYw70w): e
find_pano_depth, (1tWox9639W2KRHVxW45jRA): e
find_pano_id, (1.326071, 103.842835): e
find_pano_depth, (SRH5-M85aXIq3pYjmRyT1Q): e
find_pano_depth, (q4jz2AOoJzMAtSMNMrThnw): e
find_pano_depth, (4nSZ_hUGaBZE-8-zgtt3DQ): e
find_pano_id, (1.334116, 103.885599): e
find_pano_depth, (eogFXOp0BY9UbFVJrvgW9Q): e
find_pano_id, (1.361908, 103.966869): e
find_pano_id, (1.356706, 103.948468): e
find_pano_id, (1.388118, 103.834107): e
find_pano_id, (1.322161, 103.812939): e
find_pano_id, (1.386744, 103.907288): e
find_pano_id, (1.32236, 103.875477): e
find_pano_depth, (wEq_LD5zb61EaZvu7YS0pw): e
find_pano_depth, (cjqcFcgffrGv4ttLVZIPAw): e
find_pano_id, (1.338752, 103.892338): e
find_pano_id, (1.383716, 103.867723): e
find_pano_id, (1.295744, 103.774642): e
find_pano_id, (1.345765, 103.942356): e
find_pano_id, (1.316608, 103.862359): e
find_pano_depth, (1YFsg-_jfOlfHWdclcdL1Q): e
find_pano_depth, (GRoI01EMBbvkz